In [5]:

import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
from itertools import chain
from itertools import groupby
from operator import itemgetter

In [6]:
# 1. 生成候选项集
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

# 2. 计算候选项集的支持度
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can] = 1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

# 3. 生成频繁项集
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

# 4. 生成关联规则
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

# 5. 计算关联规则的置信度
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [22]:
def loadDataSet():
    data = pd.read_csv('Transactions.csv')
    # 将里面的数据转化为这种形式--[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
    # 如果这个数据是0直接删掉，如果是1替换位列的名字
    # data = data.replace(0, np.nan)
    # data = data.dropna(axis=1, how='all')
    # data = data.fillna(0)
    # data = data.values.tolist()
    # return data
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [25]:
dataSet = loadDataSet()
dataSet
# 输出 dataSet 数据类型
print(type(dataSet))
print(type(dataSet[0]))

<class 'list'>
<class 'list'>


In [12]:
# test
if __name__ == '__main__':
    dataSet = loadDataSet()
    print

    L, supportData = apriori(dataSet, minSupport=0.05)
    rules = generateRules(L, supportData, minConf=0.7)
    print(rules)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({4}) --> frozenset({1}) conf: 1.0
frozenset({4}) --> frozenset({3}) conf: 1.0
frozenset({4}) --> frozenset({1, 3}) conf: 1.0
frozenset({1, 5}) --> frozenset({2, 3}) conf: 1.0
frozenset({1, 2}) --> frozenset({3, 5}) conf: 1.0
[(frozenset({5}), frozenset({2}), 1.0), (frozenset({2}), frozenset({5}), 1.0), (frozenset({1}), frozenset({3}), 1.0), (frozenset({4}), frozenset({1}), 1.0), (frozenset({4}), frozenset({3}), 1.0), (frozenset({4}), frozenset({1, 3}), 1.0), (frozenset({1, 5}), frozenset({2, 3}), 1.0), (frozenset({1, 2}), frozenset({3, 5}), 1.0)]
